   # ------------------------- Estudios de Punto de Pedido --------------------------------------------

In [55]:
# importamos las diferentes bibliotecas con las que vamos a trajar
# Configuramos las salidas y la precisión de los datos 
import numpy as np
import pandas as pd
import sys,os
import matplotlib.pyplot as plt

pd.set_option("display.max_rows", 200, "display.max_columns",200)
np.set_printoptions(precision=1, suppress=True)

In [56]:
t=364 # días del año

# Primera parte

In [57]:
#En la primera parte se realiza la lectura de los datos y pequeñas transformaciones para su posterior uso.

In [58]:
Compras=pd.read_csv('ENTRADAS.ALMACEN.csv').rename(columns={'0':0, '2':2, '1':1, '3':3, '5':5, '5555':5555})
# Las entradas son las compras y los envíos recibidos procedentes de otras bases
# nombre de las columnas:
# 0 : producto (PN)
# 2 : punto de pedido fijado por la empresa, tambien llamado stock mínimo (SM) 
# 3 : cantidad compradas (en este caso)
# 5 : precio por unidad
# 5555: tiempo de reaprovisionamiento


In [60]:
Consumos=pd.read_csv('SALIDAS.ALMACEN.csv').rename(columns={'0':0, '1':1, '3':3})
# las salidas son la suma de los consumos y envíos a otras bases
# 0 : producto (PN)
# 1 : fecha de consumo, de 0-364 días del año
# 3 : cantidad consumida (en este caso)
# 33 : consumo medio diario
# 6 : punto de pedido
# 66. punto de pedido, más stock de seguridad
# 8: stock de seguridad (s)

In [62]:
Consumo_total=Consumos.loc[:,[0,3]].groupby([0]).sum().reset_index() # el consumo total por producto
Consumo_total[33]=Consumo_total[3]/t # añadimos una columna con el consumo medio diario de cada producto

In [63]:
numero_compras=Compras.loc[:,[0,5555]].groupby([0]).count().reset_index() # se extrae el número de veces que se ha realizado una compra

In [64]:
numero_compras_indices=Compras.loc[:,[0,5555]].groupby([0]).count() # se guardan los indices para su posterior uso


In [65]:
tiempo_medio_compras=round(Compras.loc[:,[0,5555]].groupby([0]).mean().reset_index(), 2) # el tiempo medio que tarda la compra en llegar al almacén

In [66]:
Consumos_compra=(Compras.loc[:,[0,2]].merge(Consumo_total, on=0)).merge(tiempo_medio_compras, on=0)
# se agrupan los datos de compras y consumos por producto

In [67]:
def get_safe(d):           # se define una función para obtener el stock de seguridad
    s=np.sort(d)           # esta función extrae el día en el cual ha llegado almenos el 95% de las compras de cada producto
    p=int(0.95 * len(d))
    return s[p]

In [68]:
Nivel_seguridad_95=Compras.loc[:,[0,5555]].groupby([0]).agg(get_safe).rename(columns={5555: 55}).reset_index() # se aplica el resultado de la función a los datos

In [69]:
s=Consumos_compra.merge(Nivel_seguridad_95, on=0) # se agrupan datos

In [70]:
s[8]=s[55]*s[33] # el stock de seguridad, es el resultado de multiplicar el día en el que ha llegado 
                 # el 95% de las compras de un producto por el consumo medio anualde ese mismo producto.

In [71]:
s=s.loc[:,[0,8]] # se extrae el nivel de seguridad de cada producto
s=s.drop_duplicates(subset=None,keep='first',inplace=False)

In [72]:
Consumos_compra=Consumos_compra.merge(round(Compras.loc[:,[0,5]].groupby([0]).mean().reset_index(),2),on=0)
# se incorpora el precio medio de cada productos para hacer los calculos posteriores

In [73]:
Consumos_compra=Consumos_compra.drop_duplicates(subset=None,keep='first',inplace=False)
# se elimnan dublicados

In [76]:
Consumos_compra=Consumos_compra.dropna() # se eliminan los NAs


In [77]:
Consumos_compra[Consumos_compra[0].isin([1274,1620,2033])].loc[:,[0,2,3]].rename(columns={0:'PN',2:'SM',3:'Consumo anual'})
# se muestra información de los producto que se emplean como ejemplo

,PN,SM,Consumo anual
75,1274,432.0,1570
98,1620,42.0,150
115,2033,1.0,1
